# HY2B SWH对比

In [ ]:
from HaiYangData import *
import glob

In [ ]:
# 设置空间分辨率
years = ['2019']
months = ['08']
hy_value = ['swh_ku','swh_c','rain_flag','ice_flag','surface_type']
time_window = 1800
hy_fill_value = 32767

In [ ]:
# 将WGS 84坐标（4326）转化为极射投影
crs = CRS.from_epsg(4326)
crs = CRS.from_string("epsg:4326")
crs = CRS.from_proj4("+proj=latlon")
crs = CRS.from_user_input(4326)
crs2 = CRS(proj="aeqd")

transformer = HaiYangData.set_transformer(crs,crs2)
transformer_back = HaiYangData.set_transformer(crs2,crs)

In [ ]:
hyfiles = []
for yea in years:
    for mon in months:
        hy_dir_path = r'/mnt/data12/users/hanlu/data/hy2b/ALT/'+yea + '/' + mon
        hyfiles = hyfiles + (glob.glob(hy_dir_path + '/*.nc'))
hyfiles.sort()

In [ ]:
%%time
hy_alt = HaiYangData(satellite='hy', sensor='alt',resolution=10000)
hy_ori_df = pd.DataFrame(np.column_stack((hy_alt.alt_from_nc_files(hyfiles, value=hy_value))), columns=['lon', 'lat', 'time']+hy_value)

In [ ]:
# 删除无效点,只处理北纬66°以上的数据
hy_df = hy_ori_df.drop(list(np.where(hy_ori_df['swh_ku'] == hy_fill_value)[0][:]))
hy_df = hy_alt.data_filter(hy_df,'lat',66)

In [ ]:
# 将投影数据添加到dataframe中
hy_alt.add_proj(hy_df, transformer)

# 交叉点平均化
hy_mean_grid = hy_alt.coincident_point_mean(hy_df,'swh_ku')

# 获得XYmgrid
hy_x_map, hy_y_map = hy_alt.get_map_grid(transformer_back)

In [ ]:
plt.figure(figsize=(10, 10))
hy_m = Basemap(projection='npaeqd', boundinglat=66, lon_0=180, resolution='c')
hy_m.pcolormesh(hy_x_map, hy_y_map, data=hy_mean_grid, cmap=plt.cm.jet,vmin=0, vmax=5,latlon = True)
hy_m.colorbar(location='right')
hy_m.fillcontinents()
hy_m.drawmapboundary()
hy_m.drawparallels(np.arange(-90., 120., 10.), labels=[1, 0, 0, 0])
hy_m.drawmeridians(np.arange(-180., 180., 60.), labels=[0, 0, 0, 1])
# plt.title("HY2B SWH  "+ year+ month)
plt.show()
plt.close()

In [ ]:
fig , ax = plt.subplots()
hy_df['swh_ku'].hist(ax=ax, bins=100)
ax.set_yscale('log')
plt.show()

# Cryosat 数据处理

In [ ]:
cyfiles = []
cy_fill_value = -32768

for yea in years:
    for mon in months:
        cy_dir_path = r'/mnt/data12/users/hanlu/data/cryosat2/'+yea + '/' + mon
        cyfiles = cyfiles + (glob.glob(cy_dir_path + '/*.nc'))
cyfiles.sort()

In [ ]:
%%time
cy_value = ['swh_ocean_01_ku']
cy_siral = HaiYangData(satellite='cy', sensor='siral',resolution=10000)
cy_ori_df = pd.DataFrame(np.column_stack((cy_siral.cy_siral_from_nc(cyfiles, value=cy_value))), columns=['lon', 'lat', 'time']+cy_value)

In [ ]:
# 删除无效点,只处理北纬66°以上的数据
cy_df = cy_ori_df.drop(list(np.where(cy_ori_df['swh_ocean_01_ku'] == cy_fill_value)[0][:]))
cy_df = cy_siral.data_filter(cy_df,'lat',66)

In [ ]:
fig,ax = plt.subplots()
cy_df.swh_ocean_01_ku.hist(ax=ax,bins=100)
ax.set_yscale('log')

In [ ]:
# 将投影数据添加到dataframe中
cy_siral.add_proj(cy_df, transformer)

# 交叉点平均化
cy_mean_grid = cy_siral.coincident_point_mean(cy_df,'swh_ocean_01_ku')

# 获得XYmgrid
cy_x_map, cy_y_map = cy_siral.get_map_grid(transformer_back)

In [ ]:
plt.figure(figsize=(10, 10))
cy_m = Basemap(projection='npaeqd', boundinglat=66, lon_0=180, resolution='c')
cy_m.pcolormesh(cy_x_map, cy_y_map, data=cy_mean_grid, cmap=plt.cm.jet,vmin=0, vmax=5,latlon = True)
cy_m.colorbar(location='right')
cy_m.fillcontinents()
cy_m.drawmapboundary()
cy_m.drawparallels(np.arange(-90., 120., 10.), labels=[1, 0, 0, 0])
cy_m.drawmeridians(np.arange(-180., 180., 60.), labels=[0, 0, 0, 1])
# plt.title("cy2 SWH  "+ year+ month)
plt.show()
plt.close()

# ICESAT2 数据处理

In [ ]:
%%time
# icesat2 的时间是从2018-01-01 00：00：00开始记的，Hy2b和cryosat2的是从2000-01-01 00：00：00开始计的
# 2018-01-01 00：00：00 与2000-01-01 00：00：00 相差了568080000秒
correction_second = 568080000

isfiles = []
for yea in years:
    for mon in months:
        is_dir_path = r'/mnt/data12/users/hanlu/data/icesat2/'+yea + '/' + mon
        isfiles = isfiles + (glob.glob(is_dir_path + '/*.h5'))
isfiles.sort()

In [ ]:
%%time
cy_value = ['swh_ocean_01_ku']
cy_siral = HaiYangData(satellite='cy', sensor='siral',resolution=10000)
cy_ori_df = pd.DataFrame(np.column_stack((cy_siral.cy_siral_from_nc(cyfiles, value=cy_value))), columns=['lon', 'lat', 'time']+cy_value)

In [ ]:
cy_df.min()

In [ ]:
# 删除无效点,只处理北纬66°以上的数据
cy_df = cy_ori_df.drop(list(np.where(cy_ori_df['swh_ocean_01_ku'] == cy_fill_value)[0][:]))
cy_df = cy_siral.data_filter(cy_df,'lat',66)

In [ ]:
fig,ax = plt.subplots()
cy_df.swh_ocean_01_ku.hist(ax=ax,bins=100)
ax.set_yscale('log')

In [ ]:
cy_df.swh_ocean_01_ku.min()

In [ ]:
# 将投影数据添加到dataframe中
cy_siral.add_proj(cy_df, transformer)

# 交叉点平均化
mean_grid = cy_siral.coincident_point_mean(cy_df,'swh_ocean_01_ku')

# 获得XYmgrid
cy_x_map, cy_y_map = cy_siral.get_map_grid(transformer_back)

In [ ]:
plt.figure(figsize=(10, 10))
cy_m = Basemap(projection='npaeqd', boundinglat=66, lon_0=180, resolution='c')
cy_m.pcolormesh(cy_x_map, cy_y_map, data=mean_grid, cmap=plt.cm.jet,vmin=0, vmax=5,latlon = True)
cy_m.colorbar(location='right')
cy_m.fillcontinents()
cy_m.drawmapboundary()
cy_m.drawparallels(np.arange(-90., 120., 10.), labels=[1, 0, 0, 0])
cy_m.drawmeridians(np.arange(-180., 180., 60.), labels=[0, 0, 0, 1])
# plt.title("cy2 SWH  "+ year+ month)
plt.show()
plt.close()

# 对比hy与cryosat交叉点


In [ ]:
plt.pcolor(hy_mean_grid)

In [ ]:
plt.pcolor(cy_mean_grid)

In [ ]:
# 将两个grid数组相减，还有数据的部分为交叉点
hy_cy_grid = hy_mean_grid - cy_mean_grid
hy_cy_diff_grid = np.argwhere(np.isnan(hy_cy_grid)!= True)

In [ ]:
cy_time_dict = dict(cy_siral.coincident_time_log(cy_df,value='swh_ocean_01_ku'))
hy_time_dict = dict(hy_alt.coincident_time_log(hy_df,value='swh_ku'))


In [ ]:
# 创建一个用来存储点对的list
hy_cy_coincident_list = []
hy_cy_time_list = []
for i in range(hy_cy_diff_grid.shape[0]): 
    diff_x = hy_cy_diff_grid[i][0]
    diff_y = hy_cy_diff_grid[i][1]
    if diff_y>(hy_alt.nlat / 2):
        diff_y -= hy_alt.nlat
    key = str(diff_x) + '+' + str(diff_y)
    
    for cy_time_key in cy_time_dict[key].keys():
        for hy_time_key in hy_time_dict[key].keys():
            hy_cy_time_list.append(np.abs(hy_time_key - cy_time_key))
            hy_cy_time_list.append(key)
            
            if np.abs(hy_time_key - cy_time_key)< time_window :
                coincident_value = []
                coincident_value.append(key)
                coincident_value.append(hy_time_key)
                coincident_value.append(hy_time_dict[key][hy_time_key])
                coincident_value.append(cy_time_key)
                coincident_value.append(cy_time_dict[key][cy_time_key])
                hy_cy_coincident_list.append(coincident_value)


In [ ]:
hy_cy_coincident_list

In [ ]:
hy_cy_df = pd.DataFrame(hy_cy_coincident_list, columns=['ij', 'hy_time', 'hy_value', 'other_time', 'other_value'])
hy_cy_df['diff'] = hy_cy_df['hy_value'] - hy_cy_df['other_value']

hy_cy_df_west = pd.DataFrame(hy_cy_coincident_list_west, columns=['ij', 'hy_time', 'hy_value', 'other_time', 'other_value'])
hy_cy_df_west['diff'] = hy_cy_df_west['hy_value'] - hy_cy_df_west['other_value']

In [ ]:
hy_cy_df_glob = pd.concat([hy_cy_df, hy_cy_df_west], axis=0)
print(hy_cy_df_glob.shape[0],hy_cy_df_glob['hy_value'].mean(),hy_cy_df_glob['other_value'].mean(),hy_cy_df_glob['diff'].mean(),hy_cy_df_glob['diff'].std())

In [ ]:
hy_cy_df_glob.to_csv(r'output/'+'2020'+'/year/'+str(years) +hy_value+'hy_cy_df_pol_two_year.csv')